### Run Image Through YOLO to Detect if There Is A Car In the Image Or Not

In [ ]:
#!pip install ultralytics

In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO

def detect_and_filter_cars(df, image_dir, confidence_threshold=0.5):
    """
    Detect cars in images using YOLO and flag low-confidence images.
    """
    # Load the YOLO model
    model = YOLO("/content/drive/MyDrive/Computer Vision Project/yolov8_car_detection_ks.pt")

    # Ensure output columns exist
    df['car_detected'] = False
    df['low_confidence'] = False

    low_confidence_images = []

    for idx, row in df.iterrows():
        image_path = row['image_path']

        try:
            # Run YOLO model on the image
            results = model.predict(source=image_path, verbose=False)

            # Extract detection confidences
            confidences = [box.conf.cpu().item() for box in results[0].boxes]

            # Check if any detection is above the confidence threshold
            car_detected = any(conf >= confidence_threshold for conf in confidences)
            df.loc[idx, 'car_detected'] = car_detected

            # Flag as low-confidence if all detections are below the threshold
            if not car_detected:
                df.loc[idx, 'low_confidence'] = True
                low_confidence_images.append(image_path)

            print(f"Processed: {image_path}, Car Detected: {df.at[idx, 'car_detected']}, Low Confidence: {df.at[idx, 'low_confidence']}")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

    # Save low-confidence images to a file
    with open("/content/drive/MyDrive/Computer Vision Project/Scraped_Data/low_confidence_images.txt", "w") as f:
        for image in low_confidence_images:
            f.write(image + "\n")

    print(f"Total low-confidence images: {len(low_confidence_images)}")

    return df


In [ ]:
# Path to images
image_dir = '/content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2'

# Load DataFrame
saved_df = pd.read_csv('/content/drive/MyDrive/Computer Vision Project/Scraped_Data/craigslist_labels.csv')
# test_saved_df = saved_df.head(50).copy()

# Detect cars
clean_df_with_cars = detect_and_filter_cars(saved_df, image_dir)

Streaming output truncated to the last 5000 lines.
Processed: /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2014_ford_extended_e350_super_duty_cargo_25026.jpg, Car Detected: True, Low Confidence: False
Processed: /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2014_ford_extended_e350_super_duty_cargo_25027.jpg, Car Detected: True, Low Confidence: False
Processed: /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2014_ford_extended_e350_super_duty_cargo_25028.jpg, Car Detected: True, Low Confidence: False
Processed: /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2014_ford_extended_e350_super_duty_cargo_25029.jpg, Car Detected: True, Low Confidence: False
Processed: /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2014_ford_extended_e350_super_duty_cargo_25030.jpg, Car Detected: True, Low Confidence: False
Processed: /content/drive/MyDrive/Computer Vision Proj

In [ ]:
# Save clean dataframe
clean_df_with_cars.to_csv('/content/drive/MyDrive/Computer Vision Project/Scraped_Data/craigslist_labels_clean.csv', index=False)

In [ ]:
clean_df_with_cars.shape

(29710, 10)

#### Move non-car images out of the main image folder and remove the non-car image labels from the main dataframe

In [ ]:
import os
import shutil

def move_and_separate_low_confidence_images(df, target_dir):
    """
    Move low-confidence images based on the 'low_confidence' column in the DataFrame,
    and separate those rows into a new DataFrame.
    """
    # Ensure the target directory exists
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created directory: {target_dir}")

    # Filter for low-confidence images
    low_confidence_df = df[df['low_confidence']]
    clean_df = df[~df['low_confidence']].copy()  # Copy remaining rows to avoid SettingWithCopyWarning

    # Move low-confidence images to the target directory
    for image_path in low_confidence_df['image_path']:
        try:
            shutil.move(image_path, target_dir)
            print(f"Moved: {image_path} -> {target_dir}")
        except Exception as e:
            print(f"Error moving {image_path}: {e}")

    # Return both DataFrames
    return clean_df, low_confidence_df


In [ ]:
target_directory = '/content/drive/MyDrive/Computer Vision Project/Scraped_Data/non_car_images'

# Call the move function using the DataFrame returned by detect_and_filter_cars
clean_craigslist_labels, non_car_image_labels = move_and_separate_low_confidence_images(clean_df_with_cars, target_directory)

# Save the non-car labels DataFrame to a file
non_car_image_labels.to_csv('/content/drive/MyDrive/Computer Vision Project/Scraped_Data/non_car_image_labels.csv', index=False)

Error moving /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2016_ram_4500_53.jpg: Destination path '/content/drive/MyDrive/Computer Vision Project/Scraped_Data/non_car_images/2016_ram_4500_53.jpg' already exists
Error moving /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2016_ram_4500_56.jpg: Destination path '/content/drive/MyDrive/Computer Vision Project/Scraped_Data/non_car_images/2016_ram_4500_56.jpg' already exists
Error moving /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2024_biz_on_wheels_57.jpg: Destination path '/content/drive/MyDrive/Computer Vision Project/Scraped_Data/non_car_images/2024_biz_on_wheels_57.jpg' already exists
Error moving /content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2024_biz_on_wheels_58.jpg: Destination path '/content/drive/MyDrive/Computer Vision Project/Scraped_Data/non_car_images/2024_biz_on_wheels_58.jpg' already exists
Error moving /content/dr

In [ ]:
clean_craigslist_labels.shape

(28382, 10)

In [ ]:
clean_craigslist_labels.head(30)

,year,make_model,make,model,label,image_url,listing_url,image_path,car_detected,low_confidence
0,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00m0m_aHTPc450Pm...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
1,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00Q0Q_alevaLceOv...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
2,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00x0x_e5gmtvzqsB...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
3,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00L0L_gwbjbHkcqP...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
4,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00g0g_lQyfS5TZNx...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
5,2023,ford_maverick,ford,maverick,2023_ford_maverick,https://images.craigslist.org/00I0I_257ZpvHg6K...,https://chicago.craigslist.org/wcl/cto/d/schil...,/content/drive/MyDrive/Computer Vision Project...,True,False
6,2023,ford_maverick,ford,maverick,2023_ford_maverick,https://images.craigslist.org/00C0C_e4JYTRb7HS...,https://chicago.craigslist.org/wcl/cto/d/schil...,/content/drive/MyDrive/Computer Vision Project...,True,False
7,2016,lexus_is_300,lexus,is_300,2016_lexus_is_300,https://images.craigslist.org/00M0M_9C4upf7tio...,https://chicago.craigslist.org/chc/ctd/d/villa...,/content/drive/MyDrive/Computer Vision Project...,True,False
8,2016,lexus_is_300,lexus,is_300,2016_lexus_is_300,https://images.craigslist.org/00Z0Z_1yRCticMx6...,https://chicago.craigslist.org/chc/ctd/d/villa...,/content/drive/MyDrive/Computer Vision Project...,True,False
9,2016,lexus_is_300,lexus,is_300,2016_lexus_is_300,https://images.craigslist.org/00707_fWO95qnhxq...,https://chicago.craigslist.org/chc/ctd/d/villa...,/content/drive/MyDrive/Computer Vision Project...,True,False


#### Drop row that made it through because the image had already been moved

In [ ]:
# Define the specific image path to drop
image_path_to_drop = "/content/drive/MyDrive/Computer Vision Project/Scraped_Data/car_images_v2/2015_land_rover_range_rover_sport_su_20.jpg"

# Drop the row where 'image_path' matches the given value
clean_craigslist_labels_2 = clean_craigslist_labels[clean_craigslist_labels['image_path'] != image_path_to_drop].reset_index(drop=True)

In [ ]:
clean_craigslist_labels_2.head(25)

,year,make_model,make,model,label,image_url,listing_url,image_path,car_detected,low_confidence
0,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00m0m_aHTPc450Pm...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
1,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00Q0Q_alevaLceOv...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
2,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00x0x_e5gmtvzqsB...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
3,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00L0L_gwbjbHkcqP...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
4,2014,buick_lacrosse_leather,buick,lacrosse_leather,2014_buick_lacrosse_leather,https://images.craigslist.org/00g0g_lQyfS5TZNx...,https://chicago.craigslist.org/nwi/ctd/d/highl...,/content/drive/MyDrive/Computer Vision Project...,True,False
5,2023,ford_maverick,ford,maverick,2023_ford_maverick,https://images.craigslist.org/00I0I_257ZpvHg6K...,https://chicago.craigslist.org/wcl/cto/d/schil...,/content/drive/MyDrive/Computer Vision Project...,True,False
6,2023,ford_maverick,ford,maverick,2023_ford_maverick,https://images.craigslist.org/00C0C_e4JYTRb7HS...,https://chicago.craigslist.org/wcl/cto/d/schil...,/content/drive/MyDrive/Computer Vision Project...,True,False
7,2016,lexus_is_300,lexus,is_300,2016_lexus_is_300,https://images.craigslist.org/00M0M_9C4upf7tio...,https://chicago.craigslist.org/chc/ctd/d/villa...,/content/drive/MyDrive/Computer Vision Project...,True,False
8,2016,lexus_is_300,lexus,is_300,2016_lexus_is_300,https://images.craigslist.org/00Z0Z_1yRCticMx6...,https://chicago.craigslist.org/chc/ctd/d/villa...,/content/drive/MyDrive/Computer Vision Project...,True,False
9,2016,lexus_is_300,lexus,is_300,2016_lexus_is_300,https://images.craigslist.org/00707_fWO95qnhxq...,https://chicago.craigslist.org/chc/ctd/d/villa...,/content/drive/MyDrive/Computer Vision Project...,True,False


In [ ]:
clean_craigslist_labels_2.shape

(28381, 10)

In [ ]:
# Save the cleaned car labels DataFrame to a file
clean_craigslist_labels_2.to_csv('/content/drive/MyDrive/Computer Vision Project/Scraped_Data/craigslist_labels_clean_non_cars_removed.csv', index=False)